### Importing the Environment Setting component modules

In [1]:
from irec.environment.loader.full_data import FullData

### Importing the Recommendation Agent component modules

In [2]:
from irec.recommendation.agents.simple_agent import SimpleAgent
from irec.recommendation.agents.action_selection_policies.egreedy import ASPEGreedy
from irec.recommendation.agents.value_functions.e_greedy import EGreedy

### Importing the Experimental Evaluation component modules

In [3]:
from irec.offline_experiments.evaluation_policies.fixed_interaction import FixedInteraction
from irec.offline_experiments.metric_evaluators.user_cumulative_interaction import UserCumulativeInteraction
from irec.offline_experiments.metrics.hits import Hits

### Setting the parameters

In [4]:
# Dataset
dataset = {
    'path': "./app/data/datasets/MovieLens 100k/ratings.csv",
    'random_seed': 0,
    'file_delimiter': ",",
    'skip_head': True
}
# Splitting
splitting = {'strategy': "global", 'train_size': 0.8, 'test_consumes': 5}
# Loader
loader = FullData(dataset, splitting)
train_dataset, test_dataset, _, _ = loader.process()
# Value Function
value_function = EGreedy()
# Action Selection Policy
greedy_selection = ASPEGreedy(epsilon=0.001)
# Agent
agent = SimpleAgent(value_function, greedy_selection, name="EGreedy")
# Evaluation Policy
eval_policy = FixedInteraction(num_interactions=100, interaction_size=1, save_info=True)

Applying splitting strategy: global

Test shape: (16892, 4)
Train shape: (80393, 4)


### Running and Evaluating the Performance of a MAB Model

In [5]:
import numpy as np
# Running the agent
interactions, action_info = eval_policy.evaluate(agent, train_dataset, test_dataset)
# Evaluation Setup
evaluator = UserCumulativeInteraction(
    ground_truth_dataset=test_dataset,
    num_interactions=100,
    interaction_size=1,
    interactions_to_evaluate=[5, 10, 20, 50, 100],
    relevance_evaluator_threshold=3.99
)
# Getting the results
hits_values = evaluator.evaluate(metric_class=Hits, results=interactions)

Starting EGreedy Training
Ended EGreedy Training


EGreedy: 100%|█████████████████████████████████████████████| 18900/18900 [00:01<00:00, 13574.49it/s]


Computing interaction 5 with UserCumulativeInteraction
Computing interaction 10 with UserCumulativeInteraction
Computing interaction 20 with UserCumulativeInteraction
Computing interaction 50 with UserCumulativeInteraction
Computing interaction 100 with UserCumulativeInteraction
UserCumulativeInteraction spent 0.31 seconds executing Hits metric


### Viewing the results obtained

In [6]:
print("\nResults for Hits metric:")
print("top-5:", np.mean(list(hits_values[0].values())))
print("top-10:", np.mean(list(hits_values[1].values())))
print("top-20:", np.mean(list(hits_values[2].values())))
print("top-50:", np.mean(list(hits_values[3].values())))
print("top-100:", np.mean(list(hits_values[4].values())))


Resultados para métrica de Hits:
top-5: 1.216931216931217
top-10: 2.17989417989418
top-20: 3.888888888888889
top-50: 8.455026455026456
top-100: 14.412698412698413
